In [3]:
!pip install --upgrade streamlit pyngrok pandas requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,

In [4]:
!pip install gdown

In [8]:
import gdown
import pandas as pd

# File IDs
movies_file_id = '1G2SDqc251ZjDaiQ6aLV-o80HAmgofNTe'
similarity_file_id = '1sDK-JLXLXqbCHNEm-hTK5Z4WWCgbSKqV'

movies_file_path = 'movies.pkl'
similarity_file_path = 'similarity.pkl'

# Download the files
gdown.download(f'https://drive.google.com/uc?id={movies_file_id}', movies_file_path, quiet=False)
gdown.download(f'https://drive.google.com/uc?id={similarity_file_id}', similarity_file_path, quiet=False)

# Load the .pkl files
df = pd.read_pickle(movies_file_path)
similar = pd.read_pickle(similarity_file_path)

Downloading...
From: https://drive.google.com/uc?id=1G2SDqc251ZjDaiQ6aLV-o80HAmgofNTe
To: /content/movies.pkl
100%|██████████| 2.33M/2.33M [00:00<00:00, 124MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1sDK-JLXLXqbCHNEm-hTK5Z4WWCgbSKqV
From (redirected): https://drive.google.com/uc?id=1sDK-JLXLXqbCHNEm-hTK5Z4WWCgbSKqV&confirm=t&uuid=3b41f7e6-7683-4a57-ae08-e29a064a30d8
To: /content/similarity.pkl
100%|██████████| 185M/185M [00:02<00:00, 86.9MB/s]


In [13]:
%%writefile app.py
import streamlit as st
import pandas as pd
import requests
import os

# CSS for Netflix-style background and fonts
bg_img = '''
<style>
[data-testid="stAppViewContainer"] {
    background-image: url("https://wallpapers.com/images/hd/netflix-background-gs7hjuwvv2g0e9fj.jpg");
    background-size: cover;
}

h1 {
    color: red;
    font-family: Helvetica, Arial, sans-serif;
}

h2, .stMarkdown, .stSelectbox, .stButton>button {
    color: white;
    font-family: Helvetica, Arial, sans-serif;
}

.stButton>button {
    background-color: #e50914;
    color: white;
    border: none;
    border-radius: 4px;
    padding: 10px;
    font-size: 16px;
    font-weight: bold;
    cursor: pointer;
}

.stButton>button:hover {
    background-color: #f40612;
}

label {
    color: white;
}

.stSelectbox {
    color: white;
}


[data-testid="stSelectbox"] label {
    color: white;
}


</style>
'''
st.markdown(bg_img, unsafe_allow_html=True)

def fetch_poster(movie_id):
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key=1567e54c5f3a5de2b7d2d7a97e3383c3&language=en-US'.format(movie_id))
    data = response.json()
    return "https://image.tmdb.org/t/p/w500/" + data['poster_path']

def recommend(movie):
    movies_index = df[df['title'] == movie].index[0]
    distances = similar[movies_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])

    recommended_movies = []
    recommended_movie_posters = []
    for i in movies_list[1:6]:
        movie_id = df.iloc[i[0]].movie_id

        recommended_movies.append(df.iloc[i[0]].title)
        # Fetching movie posters from API
        recommended_movie_posters.append(fetch_poster(movie_id))
        colored_titles = [f'<span style="color:white">{title}</span>' for title in recommended_movies]

    return colored_titles, recommended_movie_posters

df = pd.read_pickle('/content/movies.pkl')
similar = pd.read_pickle('/content/similarity.pkl')

st.title("Netflix Movies")
st.write("Welcome to the Netflix Movies!")

select_movie = st.selectbox('Search A Movie', df['title'].values, index=0, key='select_movie', help="Select a movie to get recommendations.")

if st.button('More Like This'):
    names, posters = recommend(select_movie)
    poster_width = 140

    col1, col2, col3, col4, col5 = st.columns(5)
    for i, col in enumerate([col1, col2, col3, col4, col5]):
        with col:
            st.markdown(names[i], unsafe_allow_html=True)
            st.image(posters[i], width=poster_width)

Overwriting app.py


In [10]:
!ngrok authtoken 2kxj4oFdLOksiFfTOhPLaRV2Xta_4cbC5FHWqfQiHpUaeFyRP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
!streamlit run app.py --server.port 8502 &>/dev/null &

In [12]:
from pyngrok import ngrok
public_url = ngrok.connect(8502, 'http')
print('Public URL:', public_url)

Public URL: NgrokTunnel: "https://25c0-34-73-184-39.ngrok-free.app" -> "http://localhost:8502"
